# Turing

In [1]:
using Distributions, Turing

In [2]:
ys = [0, 1, 0, 1, 0, 0, 0, 0, 0, 1]

@model bernoulli begin
    θ ~ Beta(1, 1)
    for y in ys
        y ~ Bernoulli(θ)
    end
    @predict θ
end

bernoulli (generic function with 4 methods)

In [3]:
chain = sample(bernoulli, Turing.HMC(2400, 0.3, 2))

[HMC]: Finshed with accept rate = 0.9820833333333333 within 6.133347034454346 seconds


Chain, model evidence (log)  0.0 and means Dict(:θ=>0.332075)

In [5]:
using Mamba

In [10]:
summarystats(Chains(chain[:θ]))

          Mean        SD       Naive SE       MCSE        ESS  
Param1 0.33207503 0.13257444 0.0027061643 0.0056754616 545.6538



# Stan

In [11]:
using Mamba, Stan

In [12]:
bernoullimodel = "
data { 
  int<lower=1> N; 
  int<lower=0,upper=1> y[N];
} 
parameters {
  real<lower=0,upper=1> theta;
} 
model {
  theta ~ beta(1,1);
  y ~ bernoulli(theta);
}
"

bernoullidata = [Dict("N" => 10, "y" => [0, 1, 0, 1, 0, 0, 0, 0, 0, 1])]

stanmodel = Stanmodel(update=1200, thin=2, name="bernoulli", model=bernoullimodel, nchains=1);

In [16]:
ProjDir = dirname(@__FILE__)

sim = stan(stanmodel, bernoullidata, ProjDir, CmdStanDir=CMDSTAN_HOME)


make: `/Users/kai/Turing/TuringDemo.jl/Bernoulli/tmp/bernoulli' is up to date.

Inference for Stan model: bernoulli_model
1 chains: each with iter=(1200); warmup=(0); thin=(1); 1200 iterations saved.

Warmup took (0.011) seconds, 0.011 seconds total
Sampling took (0.021) seconds, 0.021 seconds total

                Mean     MCSE   StdDev    5%   50%   95%  N_Eff  N_Eff/s    R_hat
lp__            -8.2  3.0e-02  6.2e-01  -9.4  -7.9  -7.6    428    20449  1.0e+00
accept_stat__   0.93  3.0e-03  1.0e-01  0.71  0.97   1.0   1179    56354  1.0e+00
stepsize__      0.98  7.8e-15  5.6e-15  0.98  0.98  0.98   0.50       24  1.0e+00
treedepth__      1.7  1.7e-02  5.3e-01   1.0   2.0   2.0   1034    49431  1.0e+00
n_leapfrog__     2.5  3.8e-02  1.3e+00   1.0   3.0   3.0   1145    54743  1.0e+00
n_divergent__   0.00  0.0e+00  0.0e+00  0.00  0.00  0.00   1200    57375      nan
theta           0.33  6.6e-03  1.3e-01  0.13  0.32  0.54    389    18582  1.0e+00

Samples were drawn using hmc with nuts.


Object of type "Mamba.Chains"

Iterations = 1:2399
Thinning interval = 2
Chains = 1
Samples per chain = 1200

[-9.06319 1.0 … 0.0 0.140408; -7.88965 0.833353 … 0.0 0.242368; … ; -7.64733 1.0 … 0.0 0.351918; -7.78097 0.958485 … 0.0 0.263641]